# database
> Using Pydantic's BaseSettings object to manage SQLAlchemy Database engines.

In [ ]:
#| default_exp database

In [ ]:
#| exporti

from humble_database.utils import delegates

## SQL Alchemy version
import sqlalchemy
if int(sqlalchemy.__version__[0]) < 2:
    
    from sqlalchemy.engine.url import URL
    from sqlalchemy.engine import create_engine,Engine
    from snowflake.sqlalchemy import URL as SnowflakeURL
else:
    from sqlalchemy import URL
    from sqlalchemy import create_engine, Engine
    
from sqlalchemy.orm import Session
from pydantic import SecretStr, Field
from pydantic_settings import BaseSettings, SettingsConfigDict
from typing import Optional,Union
from abc import ABC, abstractproperty,abstractmethod
from contextlib import contextmanager
import pandas as pd
from sqlalchemy import text

In [ ]:
#| hide 

from nbdev.showdoc import show_doc
import os

## SQL Alchemy Connection
> To create our Database connections, we will use SQL Alchemy's `URL` object

The following is from the SQL Alchemy fucntional documentation:

In [ ]:
show_doc(URL.create)

---

### URL.create

>      URL.create (drivername:str, username:Optional[str]=None,
>                  password:Optional[str]=None, host:Optional[str]=None,
>                  port:Optional[int]=None, database:Optional[str]=None, query:M
>                  apping[str,Union[Sequence[str],str]]=immutabledict({}))

Create a new :class:`_engine.URL` object.

.. seealso::

    :ref:`database_urls`

:param drivername: the name of the database backend. This name will
  correspond to a module in sqlalchemy/databases or a third party
  plug-in.
:param username: The user name.
:param password: database password.  Is typically a string, but may
  also be an object that can be stringified with ``str()``.

  .. note::  A password-producing object will be stringified only
     **once** per :class:`_engine.Engine` object.  For dynamic password
     generation per connect, see :ref:`engines_dynamic_tokens`.

:param host: The name of the host.
:param port: The port number.
:param database: The database name.
:param query: A dictionary of string keys to string values to be passed
  to the dialect and/or the DBAPI upon connect.   To specify non-string
  parameters to a Python DBAPI directly, use the
  :paramref:`_sa.create_engine.connect_args` parameter to
  :func:`_sa.create_engine`.   See also
  :attr:`_engine.URL.normalized_query` for a dictionary that is
  consistently string->list of string.
:return: new :class:`_engine.URL` object.

.. versionadded:: 1.4

    The :class:`_engine.URL` object is now an **immutable named
    tuple**.  In addition, the ``query`` dictionary is also immutable.
    To create a URL, use the :func:`_engine.url.make_url` or
    :meth:`_engine.URL.create` function/ method.  To modify a
    :class:`_engine.URL`, use the :meth:`_engine.URL.set` and
    :meth:`_engine.URL.update_query` methods.

## Database Settings
> A Pydantic model that controls argumetns passed to the SQL alchemy URL constructor

In [ ]:
#| exporti

class DatabaseSettingsBase(BaseSettings):
    username: Optional[str]=None
    password: Optional[SecretStr]=None

In [ ]:
#| exporti

class DatabaseSettings(DatabaseSettingsBase):
    drivername:str 
    host: Optional[str]=None
    port: Optional[int]=None
    database: Optional[str]=None
    query: dict[str,str]={}
    

In [ ]:
#| exporti 

class SnowflakeSettings(DatabaseSettingsBase):

    """Passed to special Snowflake URL constructor"""
    model_config = SettingsConfigDict(env_prefix='snowflake_',env_file = '.env')
    
    account: str = 'rentprogress'
    database: Optional[str] = 'TRANSFORM_PROD'
    snowflake_schema: Optional[str] = Field(
        'EDW',
        env='snowflake_schema',
        alias='schema',
        description="""passed to the SQLAlchemy URL constructor as 'schema', 
        but we need to alias it to avoid clashing with 'BaseSettings'""")
    warehouse: Optional[str] = 'ADHOC_WH'
    role: Optional[str] = 'DATA_DEVELOPER'
    numpy: Optional[bool] = True

    

In [ ]:
settings = DatabaseSettings(
    drivername='sqlite',
    database='test.db',
)
settings

DatabaseSettings(username=None, password=None, drivername='sqlite', host=None, port=None, database='test.db', query={})

In [ ]:
url = URL.create(
    **settings.model_dump()
)
engine = create_engine(url)
engine

Engine(sqlite:///test.db)

#### Why use Pydantic for Database Settings? 
> Pydantic's `BaseSettings` object comes with support for environment variables and secret strings

In [ ]:
# set some environment variables
os.environ['username']='humble_chuck'
os.environ['password']='super secret passkey'

# pydantic automatically reads them
settings = DatabaseSettings(drivername='mysql')
assert settings.username == 'humble_chuck'

# pydantic also keeps your password safe from displays
print(f"password prints as a secret: {settings.password}")

# by default, __init__ values take precedence over environment variables
alternate_settings = DatabaseSettings(drivername='mysql',username='some_other_person')
assert alternate_settings.username == 'some_other_person'

for k in ('username','password'):
    os.environ.pop(k)

password prints as a secret: **********


In [ ]:
#| export

class AbstractDatabaseClass(ABC):

    """
    Abstract Base Class used to define methods for all Database Connections. 

    A seperate Base Class is required because Snowflake Database Settings will behave differently, and thus they will need their own Database Settings class.
    """

    def query_to_records(
        self,
        query_string:str,
    ):
        with self._engine.connect() as conn:
            results = [row for row in conn.execute(text(query_string)).mappings()]
        return results

    @delegates(pd.read_sql_query)
    def query_to_df(
        self,
        query_string,
        **kwargs
    ):
        f"""{pd.read_sql_query.__doc__}"""
        with self._engine.connect() as conn:
            df = pd.read_sql_query(query_string,conn,**kwargs)
        return df

    @contextmanager
    def engine_scope(self,**kwargs):
        with self._engine.connect() as conn: 
            try:
                yield conn
                conn.commit()
            except:
                raise
            finally:
                conn.close()
                
    @contextmanager
    def session_scope(self,bind=None,**kwargs):
        """Provide a transactional scope around a series of operations."""

        session = Session(bind=self._engine,**kwargs)
        try:
            yield session
            session.commit()
        except:
            session.rollback()
            raise
        finally:
            session.close()


In [ ]:
#| export 

class Database(DatabaseSettings,AbstractDatabaseClass):
    """
    Create a Database connection with default functionality. 
    
    Inherits attributes from `DatabaseSettings` to manage credentials. Inherits methods from `AbstractDatabaseClass`.

    Use the class as-is to quickly create a database connection, or create a subclass to control connections to a specific database.
    
    More on SQL Alchemy engines from [SQLAlchemy docs](https://docs.sqlalchemy.org/en/13/core/engines.html):
    
    - The Engine is the starting point for any SQLAlchemy application. It’s “home base” for the actual database and its DBAPI.  
    - An Engine references both a Dialect and a Pool, which together interpret the DBAPI’s module functions as well as the behavior of the database 
    - Pool object which will establish a DBAPI connection at localhost:5432 when a connection request is first received  
    -- Note that the Engine and its underlying Pool do **not** establish the first actual DBAPI connection until the Engine.connect() method is called, or an operation which is dependent on this method such as Engine.execute() is invoked.
    - In this way, Engine and Pool can be said to have a lazy initialization behavior.  
    - The Engine, once created, can either be used directly to interact with the database, or can be passed to a Session object to work with the ORM.  
    """
    _engine:Engine = None
    _engine_url:URL = None

    def __init__(
        self,
        **kwargs
    ):
        # settings __init__
        super().__init__(**kwargs)
        if hasattr(self.password,'get_secret_value'):
            
            password = self.password.get_secret_value()
            
        else:
            password = self.password
        url = URL.create(
            drivername=self.drivername,
            username=self.username,
            password=password,
            host=self.host,
            port=self.port,
            database=self.database,
            query=self.query
        )
        self._engine_url=url
        self._engine=create_engine(url)

    
    model_config = SettingsConfigDict(
        #allows for attributes of `database settings` to be set as defaults in subclasses without type annotation
        ignored_types=(int,str,dict),
        arbitrary_types_allowed=True
    )
    

In [ ]:
#| export 

class Snowflake(SnowflakeSettings,AbstractDatabaseClass):
    """
    Create a Database connection with default functionality. 
    
    Inherits attributes from `DatabaseSettings` to manage credentials. Inherits methods from `AbstractDatabaseClass`.

    Use the class as-is to quickly create a database connection, or create a subclass to control connections to a specific database.
    
    More on SQL Alchemy engines from [SQLAlchemy docs](https://docs.sqlalchemy.org/en/13/core/engines.html):
    
    - The Engine is the starting point for any SQLAlchemy application. It’s “home base” for the actual database and its DBAPI.  
    - An Engine references both a Dialect and a Pool, which together interpret the DBAPI’s module functions as well as the behavior of the database 
    - Pool object which will establish a DBAPI connection at localhost:5432 when a connection request is first received  
    -- Note that the Engine and its underlying Pool do **not** establish the first actual DBAPI connection until the Engine.connect() method is called, or an operation which is dependent on this method such as Engine.execute() is invoked.
    - In this way, Engine and Pool can be said to have a lazy initialization behavior.  
    - The Engine, once created, can either be used directly to interact with the database, or can be passed to a Session object to work with the ORM.  
    """
    _engine:Engine = None
    _engine_url:URL = None

    def __init__(
        self,
        **kwargs
    ):
        # settings __init__
        super().__init__(**kwargs)
        if hasattr(self.password,'get_secret_value'):
            
            password = self.password.get_secret_value()
            
        else:
            password = self.password
        url = SnowflakeURL(
            user=self.username,
            password=password,
            account=self.account,
            warehouse=self.warehouse,
            database=self.database,
            schema = self.snowflake_schema,
            numpy=self.numpy
        )
        self._engine_url=url
        self._engine=create_engine(url)

    
    model_config = SettingsConfigDict(
        #allows for attributes of `database settings` to be set as defaults in subclasses without type annotation
        ignored_types=(int,str,dict),
        arbitrary_types_allowed=True,
        
    )

## Examples:

To use the `Database` as-is, pass the desired variables to the constructor: 

In [ ]:
db = Database(drivername='sqlite',database='test.db')
print(db)

username=None password=None drivername='sqlite' host=None port=None database='test.db' query={}


In [ ]:
with db._engine.connect() as conn: 
    print(type(conn.connection))
    print(type(conn))
    print(hasattr(conn,'commit'))
    

<class 'sqlalchemy.pool.base._ConnectionFairy'>
<class 'sqlalchemy.engine.base.Connection'>
True


In [ ]:
users = pd.DataFrame({
    'id':[1,2,3],
    'user':['larry','moe','curly']
})

with db.engine_scope() as conn:
    users.to_sql('users',conn,if_exists='replace',index=False)

queried = db.query_to_df("select * from users")

assert queried.equals(users)
os.remove('test.db')

### Create a sub-class to handle connections to a specific database

In [ ]:
class RNACentralDatabase(Database):
    model_config = SettingsConfigDict(env_prefix='rna_db_')
    
    drivername='postgresql'
    database='pfmegrnargs'
    host='hh-pgsql-public.ebi.ac.uk'
    port=5432
    

os.environ['rna_db_username']='reader'
os.environ['rna_db_password']='NWDMCE5xdipIjRrp'

rna_db = RNACentralDatabase()
rna_db

RNACentralDatabase(username='reader', password=SecretStr('**********'), drivername='postgresql', host='hh-pgsql-public.ebi.ac.uk', port=5432, database='pfmegrnargs', query={})

In [ ]:
rna_db.query_to_df("""select * from rnc_database limit 5""")

,id,timestamp,userstamp,descr,current_release,full_descr,alive,for_release,display_name,project_id,avg_length,min_length,max_length,num_sequences,num_organisms
0,5,2017-05-17,RNACEN,VEGA,98,VEGA,N,None,VEGA,PRJEB4568,NaN,NaN,NaN,0,0
1,1,2017-05-01,RNACEN,ENA,731,ENA,Y,None,ENA,None,412.0,10.0,900074.0,12086180,814855
2,26,2017-05-01,RNACEN,GENCODE,450,GENCODE,N,None,GENCODE,None,889.0,32.0,205012.0,47677,2
3,3,2017-05-01,RNACEN,SRPDB,732,SRPDB,Y,None,SRPDB,PRJEB4384,173.0,30.0,533.0,503,684
4,15,2017-05-02,RNACEN,WORMBASE,735,WormBase,Y,None,WormBase,PRJNA13758,174.0,17.0,84141.0,26116,1


In [ ]:
rna_db.query_to_records("select id,timestamp from rnc_database limit 1")[0]

{'id': 5, 'timestamp': datetime.datetime(2017, 5, 17, 0, 0)}

In [ ]:
with rna_db.engine_scope() as conn: 
    result = conn.execute(text("""select * from rnc_database limit 5"""))
    print(result.fetchall()[0])

(5, datetime.datetime(2017, 5, 17, 0, 0), 'RNACEN', 'VEGA', 98, 'VEGA', 'N', None, 'VEGA', 'PRJEB4568', None, None, None, 0, 0)


In [ ]:
#| hide

!nbdev_export